# LSH（Locality-Sensitive Hashing）

## 核心思想
目的：在高维空间中快速找到“相似”的数据点。

关键思想：让相似的样本在哈希后 落入同一个桶（bucket） 的概率高，不相似的样本落入同桶的概率低。


## 算法原理

### 1. 构建随机超平面（Random Hyperplanes）

在d维空间随机生成k个向量 (r1, r2, ... , rk)  shape(k, d)

每个向量的分布服从标准正态分布N(0,1)

### 2. 计算Hash签名（Hash Signature）

对于一个向量x, 计算h(x) -> 1/0

h_i(x) = 1 if r_i * x >= 0

x与超平面相乘，如果为整数，说明两个向量角度相似，返回1

每个超平面的结果拼接成一个二进制串（如 10100110）， 这就是该向量的 哈希签名

### 3. 构建多个Hash表

有L组Hash表

每组有k个hash函数组成一个hash tablc 

每个样本被插入到 L 个不同表中，从而提升召回率。

k = 10~20
L = 20~100 

### 4. 查询(Query)

给定查询向量q

1. 计算其在每个哈希表中的签名（每个hash表有k个hash函数）
2. 找出所有桶中与其哈希相同的候选样本
3. 对候选样本计算真实相似度（如余弦或欧式距离）c
4. 返回最相似的 Top-K

### 5. 算法复杂度

建表： 时间复杂度O(n*k*L)
查询： 时间复杂度O(L*(k+c))


In [1]:
import numpy as np
from typing import List, Union, Dict

In [33]:
class CosineLSH:
    '''
    基于 随机超平面投影 的 局部敏感哈希(LSH) 
    适用于余弦相似度测量

    参数:
    - hash_size(int): 单个哈希表的哈希函数数量
    - num_tables(int): 使用的哈希表数量
    '''
    def __init__(self, hash_size: int=6, num_tables: int=5, dim=None) -> None:
        self.hash_size = hash_size      # 每个哈希表的位数
        self.num_tables = num_tables    # 哈希表数量
        self.hash_tables = [dict() for _ in range(num_tables)]
        self.random_planes_list = []    # 存储每个哈希表的随机超平面
        self.dimension = dim           # 数据维度（在插入数据时确定）

    def _generate_random_planes(self, dimension: int) -> np.ndarray:
        '''
        为每个哈希表生成随机超平面（每一个超平面对应一个哈希函数）
        '''
        return np.random.randn(self.hash_size, dimension) # shape(hash_size, dimension)
  
    def _hash(self, vector: np.ndarray, random_planes: np.ndarray) -> str:
        '''
        计算单个向量的hash值（二进制字符串）
        '''
        projections = np.dot(vector, random_planes.T)
        hash_bits = (projections > 0).astype(int) # 大于0->1, 否则为0
        return ''.join(hash_bits.astype(str))

    def index(self, data) -> None:
        '''
        将数据向量插入到LSH索引中
        
        参数:
        - data: 待索引的向量列表
        '''
        data_array = np.array(data)
        if len(data_array.shape) == 1:
            data_array = data_array.reshape(1, -1) # shepe(dim,0) -> shape(1,dim) 将一个向量转成一个1行矩阵

        self.dimension = data_array.shape[1]

        # 为每一个hash table生成随机超平面
        self.random_planes_list = [
            self._generate_random_planes(self.dimension)
            for _ in range(self.num_tables)
        ]

        # 将每一个向量插入所有hash table
        for i, vector in enumerate(data_array):
            for table_idx in range(self.num_tables):
                hash_key = self._hash(vector, self.random_planes_list[table_idx])

                # 将向量索引存到对应的hash桶
                if hash_key in self.hash_tables[table_idx]:
                    self.hash_tables[table_idx][hash_key].append(i)
                else:
                    self.hash_tables[table_idx][hash_key] = [i]

    def query(self, query_vector, max_results = 10) -> List[int]:
        '''
        查询与给定向量相似的向量

        参数:
        - query_vector: 查询向量
        - max_results: 返回的最大结果数量 top_k

        返回:
        - 相似向量的索引列表
        '''
        if self.dimension is None:
            raise ValueError('Please Insert Data')

        query_vec = np.array(query_vector)
        candidates = set()
        
        # 在所有哈希表中查找候选向量
        for table_idx in range(self.num_tables):
            hash_key = self._hash(query_vec, self.random_planes_list[table_idx])
            if hash_key in self.hash_tables[table_idx]:
                candidates.update(self.hash_tables[table_idx][hash_key])

        # 如果没有找到候选向量，尝试查找邻近桶
        if not candidates:
            print('No exact matching candidate vector found; searching neighboring buckets....')

            for table_idx in range(self.num_tables):
                original_key = self._hash(query_vec, self.random_planes_list[table_idx])
                # 查找哈希码只有1位不同的桶
                for i in range(self.hash_size):
                    neighbor_key = list(original_key)
                    neighbor_key[i] = '1' if neighbor_key[i] == '0' else '0'
                    neighbor_key = ''.join(neighbor_key)
                    if neighbor_key in self.hash_tables[table_idx]:
                        candidates.update(self.hash_tables[table_idx][neighbor_key])

        return list(candidates)[:max_results]

    def get_hash_tables_info(self) -> Dict:
        '''
        返回哈希表统计信息
        '''
        info = {
            'num_tables': self.num_tables,
            'hash_size': self.hash_size,
            'total_buckets': 0,
            'average_bucket_size': 0,
            'table_details': [],
            'total_vectors': 0
        }

        total_vectors = 0
        for i, table in enumerate(self.hash_tables):
            num_buckets = len(table)  # 有多少个hash_key
            vectors_in_table = sum(len(bucket) for bucket in table.values()) # sum(每个key有多少index)
            total_vectors += vectors_in_table

            average_bucket_size = vectors_in_table / num_buckets if num_buckets > 0 else 0
            info['table_details'].append({
                'table_index': i,
                'num_buckets': num_buckets,
                'total_vectors': vectors_in_table,
                'average_bucket_size': average_bucket_size
            })

        info['total_vectors'] = total_vectors
        info['total_buckets'] = sum(detail['num_buckets']
            for detail in info['table_details']
        )

        if info['total_buckets'] > 0:
            info['average_bucket_size'] = (total_vectors /info['total_buckets'])

        return info


In [ ]:
def run_cosine_hash():
    np.random.seed(42)
    data_vectors = np.random.randn(100, 10) # 100个10维向量 (100, 10)

    # 创建LSH索引
    print('creating LSH index')
    lsh = CosineLSH(hash_size=8, num_tables=3)
    lsh.index(data_vectors)

    # 显示hash tables 统计信息
    info = lsh.get_hash_tables_info()
    print(f'\n哈希表统计信息:')
    print(f'哈希表数量: {info['num_tables']}')
    print(f'总桶数: {info['total_buckets']}')
    print(f'平均每个桶的向量数: {info['average_bucket_size']:.2f}')

    # 查询
    query_vec = data_vectors[0]
    print(f'\n查询向量索引: 0')

    similar_indices = lsh.query(query_vec, max_results=5)
    print(f'找到的相似向量索引: {similar_indices}')

    # 验证结果：计算实际余弦相似度
    from sklearn.metrics.pairwise import cosine_similarity
    
    print('\n相似度验证:')
    for idx in similar_indices:
        similarity = cosine_similarity([query_vec], [data_vectors[idx]])[0][0]
        print(f'向量 {idx} 与查询向量的余弦相似度: {similarity:.4f}')

    # 对比线性搜索结果
    print('\n=== 与线性搜索对比 ===')
    all_similarities = cosine_similarity([query_vec], data_vectors)[0]
    top_linear = np.argsort(all_similarities)[::-1][1:6]  # 排除自身，取前5个
    print(f'线性搜索Top-5结果: {top_linear}')

    # 计算召回率
    lsh_recall = len(set(similar_indices) & set(top_linear)) / len(top_linear)
    print(f'LSH召回率（与真实Top-5相比）: {lsh_recall:.2%}')


In [25]:
run_cosine_hash()

creating LSH index

哈希表统计信息:
哈希表数量: 3
总桶数: 189
平均每个桶的向量数: 1.59

查询向量索引: 0
找到的相似向量索引: [0, 48, 20, 54, 86]

相似度验证:
向量 0 与查询向量的余弦相似度: 1.0000
向量 48 与查询向量的余弦相似度: -0.2653
向量 20 与查询向量的余弦相似度: 0.5041
向量 54 与查询向量的余弦相似度: 0.4609
向量 86 与查询向量的余弦相似度: 0.6143

=== 与线性搜索对比 ===
线性搜索Top-5结果: [91 32 15 86 20]
LSH召回率（与真实Top-5相比）: 40.00%


理解LSH算法的参数对掌握其工作原理至关重要：

1. hash_size（hash码长度）

决定每个哈希表的哈希码长度（位数）影响：

值越大，哈希桶划分越精细，相似度判断越准确，但每个桶内的向量可能越少

2. num_tables (hash表数量)

控制使用的独立哈希表数量 影响：值越大，找到真正近邻的概率越高，但内存消耗也会增加
